In [1]:
#IMPORT LIBRARIES

import pymysql as mysql
import pandas as pd
import psycopg2 as pg2


In [2]:
#CONNECTING TO THE PEOPLE PLATFORM ID TABLE

db_connection = mysql.connect( host='eloomi-global.mysql.database.azure.com',  database='eloomi_global', user='emir@eloomi-global', password='jAU9qHUz9MuK2ejTern5')

In [3]:
db_cursor = db_connection.cursor()


In [4]:
db_cursor.execute('SELECT * FROM platforms_domains')


6949

In [5]:
db_platform_id = db_cursor.fetchall()
cols = []

for i in db_cursor.description:
    cols.append(i[0])

db_platform_id = pd.DataFrame(db_platform_id, columns = cols)  

In [6]:
db_platform_id.head(5)

,domain_id,platform_id,domain,created_at,updated_at
0,10,3.0,ricksplat.eloomi.com,NaT,NaT
1,11,2.0,kennethstest.eloomi.com,NaT,NaT
2,12,4.0,demov2.eloomi.com,NaT,NaT
3,13,5.0,dermanorakademiet.eloomi.com,NaT,NaT
4,14,6.0,david-andersen.eloomi.com,NaT,NaT


In [7]:
#### END #####

In [8]:
#CONNECTING TO THE INFINITE ID TABLE

In [9]:
pip install pymssql

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pymssql

conn = pymssql.connect(
         server="eloomi-mssql-server-prod.database.windows.net",
         port=1433,
         user="emir",
         password="jAU9qHUz9MuK2ejTern5",
         database="eloomi-mssql-db-global-platforms-prod")
conn
cursor = conn.cursor()
cursor.execute("SELECT * FROM platform_domains")

In [11]:
db_infinite_cursor = conn.cursor()  


In [12]:
db_infinite_cursor.execute('SELECT * FROM platform_domains')


In [13]:
db_infinite_platform_id = db_infinite_cursor.fetchall()
cols = []

for i in db_infinite_cursor.description:
    cols.append(i[0])

db_infinite_platform_id = pd.DataFrame(db_infinite_platform_id, columns = cols)  

In [14]:
db_infinite_platform_id.columns

columns_to_keep = ['id', 'platform_id','domain']
db_infinite_platform_id_new = db_infinite_platform_id[columns_to_keep]


In [15]:
db_infinite_platform_id_new.head(5)

,id,platform_id,domain
0,1,1177,external.eloomi.io
1,2,1348,learn.eloomi.com-deleted
2,3,1350,eloomi.eloomi.com-deleted
3,4,1428,tutby1.eloomi.com
4,5,1429,my-comp.eloomi.com


In [16]:
import pandas as pd
import snowflake.connector

def send_df_to_snowflake(df, table_name, schema, database, user, password, account, warehouse, role):
    """
    Sends a DataFrame to Snowflake and replaces the existing table.
    
    Parameters:
    - df: pandas DataFrame to be sent to Snowflake.
    - table_name: Name of the table in Snowflake.
    - schema: Schema of the table in Snowflake.
    - database: Database of the table in Snowflake.
    - user: Snowflake user.
    - password: Snowflake password.
    - account: Snowflake account identifier.
    - warehouse: Snowflake warehouse to use.
    - role: Snowflake role to use.
    """
    
    # Establish Snowflake connection
    conn = snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema,
        role =role
    )
    
    # Create a cursor object
    cur = conn.cursor()
    
    temp_csv = 'temp_file.csv'
    df.to_csv(temp_csv, index=False)
    
    # Use Snowflake PUT command to upload the CSV file to a Snowflake stage
    cur.execute(f"PUT file://{temp_csv} @%{table_name}")
    
    # Delete existing table data
    cur.execute(f"TRUNCATE TABLE {schema}.{table_name}")
    
    # Load data from the staged CSV file into the Snowflake table
    cur.execute(f"""
        COPY INTO {schema}.{table_name}
        FROM @%{table_name}/{temp_csv.split('/')[-1]}
        FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"')
        ON_ERROR = 'CONTINUE'
    """)
    
    # Remove the staged file
    cur.execute(f"REMOVE @%{table_name}/{temp_csv.split('/')[-1]}")
    
    # Close the cursor and connection
    cur.close()
    conn.close()

In [17]:
# Send DataFrame to Snowflake
send_df_to_snowflake(
        df=db_infinite_platform_id_new,
        table_name='INFINITE_PLATFORM_ID',
        schema='PLATFORM_DATA',
        database='RAW_DB',
        user='emirkabasoglu',
        password='7Bb13287',
        account='se63477.north-europe.azure',
        warehouse='TRANSFORMING_WH',
        role ='ACCOUNTADMIN'
    )

In [18]:
#IMPORT LIBRARIES
#import snowflake.connector
#from snowflake.connector.pandas_tools import write_pandas
#import os
#import pandas as pd



In [19]:

# Snowflake connection parameters
#SNOWFLAKE_USER = 'emirkabasoglu'
#SNOWFLAKE_PASSWORD = '7Bb13287'
#SNOWFLAKE_ACCOUNT = 'se63477.north-europe.azure'
#SNOWFLAKE_WAREHOUSE = 'TRANSFORMING_WH'
#SNOWFLAKE_DATABASE = 'RAW_DB'
#SNOWFLAKE_SCHEMA = 'PLATFORM_DATA'

# Establish a connection to Snowflake
#conn = snowflake.connector.connect(
    #user=SNOWFLAKE_USER,
    #password=SNOWFLAKE_PASSWORD,
    #account=SNOWFLAKE_ACCOUNT,
    #warehouse=SNOWFLAKE_WAREHOUSE,
    #database=SNOWFLAKE_DATABASE,
    #schema=SNOWFLAKE_SCHEMA
#)


In [20]:
#CREATE THE ENGINE THAT CONNECTS TO THE DATABASE
#from sqlalchemy import create_engine

#engine = create_engine(f'snowflake://{SNOWFLAKE_USER}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACCOUNT}/{SNOWFLAKE_DATABASE}/{SNOWFLAKE_SCHEMA}?warehouse={SNOWFLAKE_WAREHOUSE}')

In [21]:
#SEND THE DATAFRAME TO THE DATABASE

#db_infinite_platform_id_new.to_sql('infinite_platform_id', engine,index=False,  if_exists='replace')


In [22]:
###END###

In [23]:
###REGION FOR INFINITE PLATFORMS ####

In [24]:
import pymssql

conn = pymssql.connect(
         server="eloomi-mssql-server-prod.database.windows.net",
         port=1433,
         user="emir",
         password="jAU9qHUz9MuK2ejTern5",
         database="eloomi-mssql-db-global-platforms-prod")
conn
cursor = conn.cursor()
cursor.execute("SELECT * FROM platform")

In [25]:
db_infinite_region_cursor = conn.cursor()  


In [26]:
db_infinite_region_cursor.execute('SELECT * FROM platform')


In [27]:
db_infinite_platform_region_id = db_infinite_region_cursor.fetchall()
cols = []

for i in db_infinite_region_cursor.description:
    cols.append(i[0])

db_infinite_platform_region_id = pd.DataFrame(db_infinite_platform_region_id, columns = cols)  

In [28]:
keep_columns = ['platform_id','region']
db_infinite_platform_region_id = db_infinite_platform_region_id[keep_columns]
db_infinite_platform_region_id.head(2)


,platform_id,region
0,1177,0
1,1348,0


In [29]:
#CREATE A FUNCTION TO SET UP THE REGION FOR CUSTOMERS
def map_region(region):
    if region == 1:
        return 'EU'
    elif region == 0:
        return 'US'
    else:
        return 'unknown' 

In [30]:
#ASSIGNING THE PLATFORM REGION TO THE PLATFORMS

db_infinite_platform_region_id['platform_region'] = db_infinite_platform_region_id['region'].apply(map_region)

In [31]:
# Send DataFrame to Snowflake
send_df_to_snowflake(
        df=db_infinite_platform_region_id,
        table_name='INFINITE_PLATFORM_REGION',
        schema='PLATFORM_DATA',
        database='RAW_DB',
        user='emirkabasoglu',
        password='7Bb13287',
        account='se63477.north-europe.azure',
        warehouse='TRANSFORMING_WH',
        role ='ACCOUNTADMIN'
    )

In [32]:
#SEND THE DATAFRAME TO THE DATABASE

#db_infinite_platform_region_id.to_sql('infinite_platform_region', engine,index=False,  if_exists='replace')
